In [1]:
import pandas as pd
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import json
from Bio import SeqIO

from scipy.stats import fisher_exact


In [2]:
exp = pd.read_csv('../ref/CpSilv_gb_annotations_and_expression_data.csv').set_index('Unnamed: 0')

In [3]:
gff_path = '../ref/genbank_files/CpSilv.genes_only.gff3'
gff = pd.read_csv(gff_path, sep='\t', header=None)[[0,2,3,4,6,8]]
gff.columns = ['chrom', 'type', 'start', 'stop', 'direction', 'annotation']
gff['gene'] = gff['annotation'].str.split(';',expand=True)[0].str.replace('ID=gene-','')

gff.head()

,chrom,type,start,stop,direction,annotation,gene
0,CP075068.1,gene,57375,57980,-,ID=gene-D8B26_000001;Name=D8B26_000001;gbkey=G...,D8B26_000001
1,CP075068.1,gene,58131,58768,-,ID=gene-D8B26_000002;Name=D8B26_000002;gbkey=G...,D8B26_000002
2,CP075068.1,gene,58628,59933,-,ID=gene-D8B26_000003;Name=D8B26_000003;gbkey=G...,D8B26_000003
3,CP075068.1,gene,60054,62173,-,ID=gene-D8B26_000004;Name=D8B26_000004;gbkey=G...,D8B26_000004
4,CP075068.1,gene,126638,126720,+,ID=gene-D8B26_000005;Name=D8B26_000005;gbkey=G...,D8B26_000005


In [9]:
gene_list = gff[(gff['chrom']=='CP075070.1') & (gff['start'].isin(range(1237751, 1428001)))]['gene'].tolist()
len(gene_list)

78

In [10]:
exp = exp[~exp['Beyhan_exp'].isna()]

In [11]:
spherule_upreg_genes = exp[(exp['Beyhan_exp']=='spherule_upreg')].index.tolist()
not_spherule_upreg_genes = exp[~(exp['Beyhan_exp']=='spherule_upreg')].index.tolist()

In [12]:
### Is the chr3 deletion region enriched for spherule upregulated genes?

chr3_spherule = len([gene for gene in gene_list if gene in spherule_upreg_genes])
chr3_not_spherule = len([gene for gene in gene_list if gene in not_spherule_upreg_genes])

not_chr3_spherule = len([gene for gene in spherule_upreg_genes if gene not in gene_list])
not_chr3_not_spherule = len([gene for gene in not_spherule_upreg_genes if gene not in gene_list])


p = fisher_exact(np.array([[chr3_spherule, chr3_not_spherule,], [not_chr3_spherule, not_chr3_not_spherule]]),
                alternative='two-sided')[1]

print(np.array([[chr3_spherule, chr3_not_spherule,], [not_chr3_spherule, not_chr3_not_spherule]]))
# print('spherule prop in category: {}, spherule prop in genome: {}'.format(round(spherule_tm/(spherule_tm+not_spherule_tm), 4),
#                                                        round(spherule_not_tm/(spherule_not_tm+not_spherule_not_tm), 4)), )
print('p = {}'.format(p))

[[  16   41]
 [1066 6178]]
p = 0.00819651590501725
